In [26]:
import openai
import tiktoken
import tempfile
import IPython
import structlog
import random
import re
import requests
import concurrent.futures
logger = structlog.getLogger()
openai.api_key_path = '/home/jong/.openai_key'

In [17]:
class Chat:
    def __init__(self, system, max_length=4096//2):
        self._system = system
        self._max_length = max_length
        self._history = [
            {"role": "system", "content": self._system},
        ]

    @classmethod
    def num_tokens_from_messages(cls, messages, model="gpt-3.5-turbo"):
        """Returns the number of tokens used by a list of messages."""
        encoding = tiktoken.encoding_for_model(model)
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
        
    def message(self, next_msg=None):
        # TODO: Optimize this if slow through easy caching
        while len(self._history) > 1 and self.num_tokens_from_messages(self._history) > self._max_length:
            logger.info(f'Popping message: {self._history.pop(1)}')
        if next_msg is not None:
            self._history.append({"role": "user", "content": next_msg})
        logger.info('requesting openai...')
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self._history,
        )
        logger.info('received openai...')
        text = resp.choices[0].message.content
        self._history.append({"role": "assistant", "content": text})
        return text

In [50]:
class EBookWriter:
    def __init__(self, topic, nchapters=16, include_code=True, theme='Sherlock Holmes mystery', theme_extra='There can be many twists to the mystery, make sure its interesting and relevant.'):
        self.topic = topic
        self.nchapters = nchapters
        self.include_code = include_code
        self.theme = theme
        self.theme_extra = theme_extra

    def get_chapters(self):
        chat = Chat('You are TextBookGPT. You come up with chapters for a textbook topic.')
        resp = chat.message(f'Write the table of contents for a textbook about {self.topic} involving {self.nchapters} chapters. Just return the ordered list of chapters and nothing else.')
        chapter_pattern = re.compile(r'\d+\.\s+.*')
        chapters = chapter_pattern.findall(resp)
        if not chapters:
            logger.warning(f'Could not parse message for chapters! Message:\n{resp}')
        return chapters
    
    def get_special_guest_for_chapter(self, chapter):
        chat = Chat(f'''You are TextBookGPT. You write chapters for textbooks on {self.topic}.
You will respond with just the name of a special guest who should appear in a chapter given to you.
Only respond with the name. Do not say anything else.''')
        return chat.message(f'Who is a good special guest for a chapter on {chapter}?')
    
    def write_chapter(self, last_chapter, curr_chapter, guest_chance=0.5):
        text = []
        system = f'''You are TextBookGPT. You write chapters for textbooks on {self.topic} in the form of a {self.theme}.
The {self.theme} must teach and be solved by {self.topic}{" code. Make sure to include code samples." if self.include_code else ""}.
{self.theme_extra}
Do not say responses to the user such as "sure".'''
        chat = Chat(system)
        msg = f'You are writing a book about {self.topic}. Write the introduction to the chapter about {curr_chapter}.'
        if last_chapter is not None:
            msg += f' Last chapter was about {last_chapter}'
        guest = None
        if random.uniform(0, 1) <= guest_chance:
            guest = self.get_special_guest_for_chapter(curr_chapter)
            msg += f' Include special guest {guest}'
        resp = chat.message(msg)
        text.append(resp)
        msg = f'Write the {self.theme} and resolution to the chapter teaching {curr_chapter}.'
        if guest is not None:
            msg += f' Include special guest {guest}'
        resp = chat.message(msg)
        text.append(resp)
        msg = f'Explain the code used to resolve the {self.theme}.'
        resp = chat.message(msg)
        text.append(resp)
        return '\n'.join(text)
        
    def run(self, nthreads=None):
        # get chapters of book
        chapters = self.get_chapters()
        # Write chapters
        text = [None] * len(chapters)
        with concurrent.futures.ThreadPoolExecutor(max_workers=nthreads or len(chapters)) as thread_pool:
            tasks = {}
            for i, prev_chapter, curr_chapter in zip(range(1_0000), [None] + chapters, chapters):
                tasks[thread_pool.submit(self.write_chapter, prev_chapter, curr_chapter)] = i
            for future in concurrent.futures.as_completed(tasks):
                idx = tasks[future]
                text[idx] = future.result()
        return chapters + text

In [49]:
e = EBookWriter('Advanced Pybind11 for Quants', nchapters=10)
book = e.run()

2023-04-12 00:12:03 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] received openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] received openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] received openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] received openai...
2023-04-12 00:12:08 [info     ] requesting openai...
2023-04-12 00:12:08 [info     ] received openai...
202

In [ ]:
themes = """
Sherlock Holmes (by Sir Arthur Conan Doyle)
Dracula (by Bram Stoker)
Frankenstein's monster (by Mary Shelley)
Robin Hood (a legendary character from English folklore)
King Arthur and the Knights of the Round Table (from various medieval texts)
Characters from Greek mythology (Zeus, Hercules, etc.)
"""
pass

In [52]:
for b in book[:12]:
    print(b)

1. Introduction to Pybind11
2. Advanced Data Types and Conversion
3. Extending C++ in Python with Pybind11
4. C++ Template Functions and Classes
5. Exception Handling and Error Reporting
6. Memory Management with Pybind11
7. Wrapping C++ Libraries with Pybind11
8. Advanced Numpy Functions with Pybind11
9. Object-Oriented Programming with Pybind11
10. Optimization Techniques in Pybind11
Introduction to Pybind11

Pybind11 is a lightweight, header-only library that that exposes C++ functions as Python modules, which can be used to interact with compiled C++ code from Python. This powerful library allows us to take advantage of the performance benefits that come with C++ while still using the python ecosystem. It is an ideal tool for quants who want to improve the performance of their python scripts.

In this chapter, we will explore Pybind11 in detail. We will discuss how Pybind11 can be integrated into existing C++ projects and show how to create Python bindings for these projects. We wi